# Задание 7. Репродуцируемость

В этом ноутбуке мы:
- фиксируем зерно случайности (seed) для всех используемых генераторов,
- выводим версии ключевых библиотек,
- выполняем небольшой детерминированный smoke-тест обучения простой модели,
  чтобы убедиться, что результаты воспроизводимы при повторном запуске.

Все сообщения и пояснения — на русском языке, код запускается без ошибок при наличии зависимостей из файла `requirements.txt`.

In [1]:
# Фиксация seed для воспроизводимости
RANDOM_SEED = 42

import os
import random
import numpy as np

def set_seed(seed: int = RANDOM_SEED) -> None:
    """
    Фиксирует источники случайности, чтобы результаты были воспроизводимыми.
    Применимо к: Python random, NumPy.
    (При необходимости можно дополнить библиотеками, если они используются и поддерживают seed.)
    """
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seed(RANDOM_SEED)
print(f"Фиксируем зерно (seed) = {RANDOM_SEED}")

Фиксируем зерно (seed) = 42


In [2]:
# Вывод версий библиотек для полной репродуцируемости окружения
import sys
import importlib

def print_version(module_name: str, alias: str | None = None):
    try:
        mod = importlib.import_module(module_name)
        ver = getattr(mod, '__version__', None)
        if ver is None:
            # Пытаемся получить версию через importlib.metadata
            try:
                from importlib.metadata import version as _v
                ver = _v(module_name)
            except Exception:
                ver = 'версия не найдена'
        name_to_show = alias if alias else module_name
        print(f"{name_to_show}: {ver}")
    except Exception as e:
        name_to_show = alias if alias else module_name
        print(f"{name_to_show}: не установлено ({e})")

print('Версии библиотек (для воспроизводимости):')
print('Python:', sys.version)
print_version('numpy', 'numpy')
print_version('pandas', 'pandas')
print_version('matplotlib', 'matplotlib')
print_version('seaborn', 'seaborn')
print_version('sklearn', 'scikit-learn')
print_version('catboost', 'catboost')
print_version('statsmodels', 'statsmodels')
# Библиотеки, которые могли использоваться ранее; не обязательны, выводим при наличии:
print_version('shap', 'shap')
print_version('xgboost', 'xgboost')
print_version('lightgbm', 'lightgbm')

Версии библиотек (для воспроизводимости):
Python: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
numpy: 2.3.5
pandas: 2.3.3
matplotlib: 3.10.7
seaborn: 0.13.2
scikit-learn: 1.7.2
catboost: 1.2.8
statsmodels: 0.14.5
shap: 0.50.0
xgboost: 3.1.2
lightgbm: не установлено (No module named 'lightgbm')


## Детерминированный smoke-тест

Ниже — короткая проверка воспроизводимости:
1) пробуем загрузить подготовленный датасет `data/interim/kc_house_data_clean.csv` (если он присутствует),
2) если файла нет, генерируем синтетические данные с фиксированным `random_state`,
3) дважды обучаем одну и ту же модель с одинаковыми настройками и seed,
4) сравниваем метрики — они должны совпасть (с точностью до очень малой погрешности вычислений).

In [3]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

set_seed(RANDOM_SEED)
csv_path = Path('..') / 'data' / 'interim' / 'kc_house_data_clean.csv'

def load_data_or_synthetic():
    if csv_path.exists():
        print(f"Загружаю датасет: {csv_path}")
        df = pd.read_csv(csv_path)
        # Определяем целевую переменную
        target_candidates = ['price', 'target', 'SalePrice']
        target = None
        for c in target_candidates:
            if c in df.columns:
                target = c
                break
        if target is None:
            # Если явную цель не нашли — берём последний числовой столбец как цель (на крайний случай)
            num_cols = df.select_dtypes(include=['number']).columns.tolist()
            if not num_cols:
                raise ValueError('В датасете нет числовых столбцов для регрессии.')
            target = num_cols[-1]
            print(f"Предупреждение: целевая переменная не найдена явно, использую столбец: {target}")
        feature_cols = [c for c in df.columns if c != target]
        X = df[feature_cols].select_dtypes(include=['number']).copy()
        # Удаляем полностью пустые/нечисловые столбцы, если такие попались
        if X.shape[1] == 0:
            raise ValueError('Не удалось собрать числовые признаки для модели.')
        y = df[target].values
        return X, y, target, X.columns.tolist()
    else:
        print('Файл с подготовленным датасетом не найден — генерирую синтетические данные.')
        from sklearn.datasets import make_regression
        X, y = make_regression(n_samples=2000, n_features=15, n_informative=10, noise=10.0, random_state=RANDOM_SEED)
        feature_cols = [f'feature_{i}' for i in range(X.shape[1])]
        return pd.DataFrame(X, columns=feature_cols), y, 'target', feature_cols

def run_once():
    X, y, target, feature_cols = load_data_or_synthetic()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_SEED
    )
    pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=1.0, random_state=None))  # Ridge детерминирован при фиксированных данных
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = float(np.sqrt(mean_squared_error(y_test, y_pred)))
    mae = float(mean_absolute_error(y_test, y_pred))
    r2  = float(r2_score(y_test, y_pred))
    return rmse, mae, r2

metrics1 = run_once()
metrics2 = run_once()
print('Повтор 1 — RMSE: {:.4f} | MAE: {:.4f} | R^2: {:.6f}'.format(*metrics1))
print('Повтор 2 — RMSE: {:.4f} | MAE: {:.4f} | R^2: {:.6f}'.format(*metrics2))
if np.allclose(np.array(metrics1), np.array(metrics2), rtol=1e-12, atol=1e-12):
    print('Результаты совпадают — воспроизводимость подтверждена.')
else:
    print('ВНИМАНИЕ: результаты отличаются — проверьте seed и окружение.')

Загружаю датасет: ..\data\interim\kc_house_data_clean.csv
Загружаю датасет: ..\data\interim\kc_house_data_clean.csv
Повтор 1 — RMSE: 141308.2895 | MAE: 100182.9957 | R^2: 0.702707
Повтор 2 — RMSE: 141308.2895 | MAE: 100182.9957 | R^2: 0.702707
Результаты совпадают — воспроизводимость подтверждена.
